In [1]:
import numpy as np
import pandas as pd 

In [11]:
events_data = pd.read_csv('https://stepik.org/media/attachments/course/4852/event_data_train.zip', compression='zip')
submission_data = pd.read_csv('https://stepik.org/media/attachments/course/4852/submissions_data_train.zip', compression='zip')
events_data['date'] = pd.to_datetime(events_data.timestamp, unit = 's')
events_data['day'] = events_data.date.dt.date

submission_data['date'] = pd.to_datetime(submission_data.timestamp, unit = 's')
submission_data['day'] = submission_data.date.dt.date

users_data = events_data.groupby('user_id', as_index= False)\
        .agg({'timestamp': 'max'})\
        .rename({'timestamp':'last_timestamp'}, axis = 'columns')
now = 1526772811
drop_out_threasold = 2592000

users_data['is_gone_user'] = (now - users_data.last_timestamp) > drop_out_threasold
users_scores = submission_data.pivot_table(index='user_id',\
                        columns = 'submission_status',\
                        values = 'step_id',\
                        aggfunc = 'count',\
                        fill_value = 0)\
                        .reset_index()
users_data = users_data.merge(users_scores, on = 'user_id', how = 'outer')
users_data = users_data.fillna(0)
users_invent_data = events_data.pivot_table(index = 'user_id',
                        columns = 'action',
                        values  = 'step_id',
                        aggfunc = 'count',
                        fill_value = 0).reset_index()
users_data = users_data.merge(users_invent_data, how = 'outer')
users_days = events_data.groupby('user_id').day.nunique()
users_days.to_frame().reset_index()
users_data = users_data.merge(users_days, on = 'user_id', how = 'outer')
users_data['passed_corse'] = users_data.passed > 170

In [12]:
users_data.head()

,user_id,last_timestamp,is_gone_user,correct,wrong,discovered,passed,started_attempt,viewed,day,passed_corse
0,1,1472827464,True,0.0,0.0,1,0,0,1,1,False
1,2,1519226966,True,2.0,0.0,9,9,2,10,2,False
2,3,1444581588,True,29.0,23.0,91,87,30,192,7,False
3,5,1499859939,True,2.0,2.0,11,11,4,12,2,False
4,7,1521634660,True,0.0,0.0,1,1,0,1,1,False


In [13]:
users_data[users_data.passed_corse].day.median()

20.0

In [14]:
user_min_time = events_data.groupby('user_id',as_index = False).agg({'timestamp':'min'}).rename({'timestamp':'min_timestamp'},axis = 1)

In [15]:
user_min_time.head()

,user_id,min_timestamp
0,1,1472827464
1,2,1514383364
2,3,1434358476
3,5,1466156809
4,7,1521634660


In [16]:
user_data = users_data.merge(user_min_time,how = 'outer')

In [17]:
event_data_train = pd.DataFrame()

In [18]:
#for user_id in users_data.user_id:
    #min_user_time = user_data[user_data.user_id == user_id].min_timestamp.item()
    #time_treshold = min_user_time + 3 * 60 * 60 * 24
    
    #user_events_data = events_data[(events_data.user_id == user_id) & (events_data.timestamp < time_treshold)]
    
    #event_data_train = event_data_train.append(user_events_data)

In [19]:
event_data_train = events_data[events_data.user_id events_data.timestamp]

SyntaxError: invalid syntax (2361919228.py, line 1)

In [22]:
events_data['user_time'] = events_data.user_id.map(str) + '_' + events_data.timestamp.map(str)

In [24]:
events_data

,step_id,timestamp,action,user_id,date,day,user_time
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15,17632_1434340848
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15,17632_1434340848
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15,17632_1434340848
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15,17632_1434340895
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15,17632_1434340895
...,...,...,...,...,...,...,...
3480698,33375,1526772685,viewed,11677,2018-05-19 23:31:25,2018-05-19,11677_1526772685
3480699,32815,1526772750,viewed,5198,2018-05-19 23:32:30,2018-05-19,5198_1526772750
3480700,32815,1526772750,discovered,5198,2018-05-19 23:32:30,2018-05-19,5198_1526772750
3480701,32815,1526772750,passed,5198,2018-05-19 23:32:30,2018-05-19,5198_1526772750


In [27]:
learning_time_treshold = 3 * 60 * 60 * 24

In [28]:
user_min_time.head()

,user_id,min_timestamp
0,1,1472827464
1,2,1514383364
2,3,1434358476
3,5,1466156809
4,7,1521634660


In [29]:
user_learning_time_treshold = user_min_time.user_id.map(str) + '_' + (user_min_time.min_timestamp + learning_time_treshold).map(str)

In [30]:
user_learning_time_treshold

0            1_1473086664
1            2_1514642564
2            3_1434617676
3            5_1466416009
4            7_1521893860
               ...       
19229    26790_1451743216
19230    26793_1510484231
19231    26794_1495546249
19232    26797_1508684516
19233    26798_1521234963
Length: 19234, dtype: object

In [35]:
event_data_train=events_data.merge(user_min_time,how='outer')
event_data_train[((event_data_train.timestamp-event_data_train.min_timestamp)<=3*24*60*60)]

,step_id,timestamp,action,user_id,date,day,user_time,min_timestamp
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15,17632_1434340848,1434340848
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15,17632_1434340848,1434340848
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15,17632_1434340848,1434340848
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15,17632_1434340895,1434340848
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15,17632_1434340895,1434340848
...,...,...,...,...,...,...,...,...
3480698,32814,1526762783,discovered,943,2018-05-19 20:46:23,2018-05-19,943_1526762783,1526762664
3480699,32814,1526762783,passed,943,2018-05-19 20:46:23,2018-05-19,943_1526762783,1526762664
3480700,32815,1526772750,viewed,5198,2018-05-19 23:32:30,2018-05-19,5198_1526772750,1526772750
3480701,32815,1526772750,discovered,5198,2018-05-19 23:32:30,2018-05-19,5198_1526772750,1526772750


In [53]:
event_data_train.groupby('user_id').day.nunique().max()

478

KeyError: "['user_learning_time_threshold'] not in index"

,step_id,timestamp,submission_status,user_id,date,day
506853,32812,1514383485,correct,2,2017-12-27 14:04:45,2017-12-27
114961,33538,1444573728,correct,3,2015-10-11 14:28:48,2015-10-11
505569,32812,1499859730,correct,5,2017-07-12 11:42:10,2017-07-12
500024,33332,1480604143,correct,8,2016-12-01 14:55:43,2016-12-01
509054,33983,1436368601,wrong,14,2015-07-08 15:16:41,2015-07-08
...,...,...,...,...,...,...
491467,32219,1469015051,correct,26787,2016-07-20 11:44:11,2016-07-20
508280,32929,1451722691,correct,26790,2016-01-02 08:18:11,2016-01-02
318273,32812,1511374868,correct,26794,2017-11-22 18:21:08,2017-11-22
506152,32812,1508425828,correct,26797,2017-10-19 15:10:28,2017-10-19


,step_id,timestamp,submission_status,user_id,date,day
0,31978,1491590313,wrong,12265,2017-04-07 18:38:33,2015-07-06
